In [1]:
import sys
sys.path.append("/playpen-ssd/smerrill/deception/BS/src")

print("✓ Added src/ to PYTHONPATH.")
from game.deck import Deck
from agents.llm_agent import LLMAgent
from simulation.bs_environment import BSEnvironment
from analysis.mc_cfe import GameRunner
#from analysis.mc_cfe import make_truthful_alternative

print("✓ Imported all modules.")

✓ Added src/ to PYTHONPATH.
✓ Imported all modules.


In [2]:
import os
from unsloth import FastLanguageModel
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"

base_model = "unsloth/Llama-3.1-8B-Instruct"
max_seq_length = 5000
device_map = {'':0}
model, tokenizer = FastLanguageModel.from_pretrained(
    base_model,
    max_seq_length=max_seq_length,
    device_map=device_map,
    load_in_4bit=True,
    fix_tokenizer=True,
    offload_folder="/playpen-ssd/smerrill/offload", 
) 

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/playpen-ssd/smerrill/conda_envs/deception/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/playpen-ssd/smerrill/conda_envs/deception/lib/python3.11/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


INFO 11-24 17:14:33 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1. vLLM: 0.11.0.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 8. Max memory: 47.438 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRM

### Single Test

In [ ]:
# Create a reproducible deck
deck = Deck(seed=123)

# Agent A prints prompts, raw outputs, parsed actions, and hand contents
agentA = LLMAgent(
    name="A",
    model=model, tokenizer=tokenizer,           # fallback simple behavior
    seed_base=1,
    debug_config={
        "show_prompts": True,
        "show_raw_output": True,
        "show_parsed_actions": True,
        "show_hands": True
    },
    log_dir="logs/notebook"
)

# Agent B prints prompts, raw outputs, parsed actions (no hand shown)
agentB = LLMAgent(
    name="B",
    model=model, tokenizer=tokenizer,
    seed_base=2,
    debug_config={
        "show_prompts": True,
        "show_raw_output": True,
        "show_parsed_actions": True
    },
    log_dir="logs/notebook"
)

# Environment prints turn state and hands
env = BSEnvironment(
    agents=[agentA, agentB],
    deck=deck,
    log_dir="logs/notebook",
    debug_config={
        "show_env_turn_state": True,
        "show_hands": True
    }
)

print("✓ Agents and environment created.")


✓ Agents and environment created.


In [ ]:
env.deal(n_cards=5)
print("✓ Dealt 5 cards to both agents.")


A hand: ['Jh', 'Ah', '9h', 'As', '7s']
B hand: ['8h', '6d', '7d', 'Th', '4c']
✓ Dealt 5 cards to both agents.


In [ ]:
print("=== Running 4 debug turns ===\n")

for i in range(4):
    snapshot = env.step(i)
    print(f"Turn {i} complete — snapshot saved to:", snapshot)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


=== Running 4 debug turns ===


--- SYSTEM + HISTORY PROMPT (agent=A, seed=1) ---
SYSTEM: You are playing the game of BS. Rules:
1. Goal: be the first to have zero cards.
2. On your turn, you must play cards matching the current required rank.
3. You may play any number of cards, and may lie about rank.
4. Your opponent may 'Challenge' if they think you are lying, or 'Pass' otherwise.
5. If challenged and you were lying, you pick up the pile; if not lying, the challenger picks up the pile.
6. If you play cards not matching the current required rank, your opponent may force you to pick up the pile.
7. Always respond in JSON exactly with keys: 'Reasoning', 'Action' ('Pass', 'Challenge', or 'PLAY'), 'Card_idx' (list of indices from your hand).
8. Include your reasoning first in the 'Reasoning' field.
SYSTEM: You are playing the card game BS.
Rules:
1. Goal: get rid of all your cards.
2. On your turn, you may play any number of cards (including 0) and declare a rank.
3. You may lie about t

### MCFE Test

In [3]:
from pathlib import Path
NUM_GAMES = 1
LOG_ROOT = "games"
N_CARDS = 5

runner = GameRunner(deck_class=Deck, agent_class=LLMAgent, log_root=LOG_ROOT)

for seed in range(NUM_GAMES):
    print(f"Running base game seed {seed:04d}")
    env, snapshots = runner.run_single_game(model, tokenizer, seed=seed, n_cards=N_CARDS)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Running base game seed 0000
--------------------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are playing the game of BS. Rules:
1. Goal: be the first to have zero cards.
2. On your turn, you may play any number of cards (including 0) and declare a rank.
3. You may lie about the rank.
4. Your opponent may Challenge (if they think you lied) or Pass.
5. If challenged and lying: you pick up the pile.
6. If challenged and truthful: challenger picks up the pile.
7. Always respond in JSON exactly with keys: 'Reasoning', 'Action' ('PLAY', 'Challenge', 'Pass'), 'Card_idx'.
8. Include your reasoning first in 'Reasoning'.<|eot_id|><|start_header_id|>user<|end_header_id|>

Game History (last 0 turns):

Game Context:
You are A.
Your hand (5 cards): 9s,5s,Kh,Qh,Jd
Opponent B has 5 cards.
Pile size: 0
Current required rank: 2
Last play: None

Respond in JSON:
{
"Reasoning": "...",
"Action": "PLAY",
"Declared_Rank": "

KeyboardInterrupt: 

In [4]:
from treys import Card
Card.int_to_str(8394515)


'9s'

In [ ]:
import json
from pathlib import Path

BASE_GAME_SEED = 42
TARGET_TURN = 5
NUM_MC = 100
LOG_ROOT = "games"

# Load snapshot for turn 5
snapshot_file = Path(LOG_ROOT) / f"game_seed_{BASE_GAME_SEED}" / "snapshots" / f"turn_{TARGET_TURN}.json"
with open(snapshot_file, "r") as f:
    snapshot = json.load(f)

runner = GameRunner(deck_class=Deck, agent_class=LLMAgent, log_root=LOG_ROOT)

mc_files = runner.run_monte_carlo(snapshot, base_seed=BASE_GAME_SEED, num_sims=NUM_MC)

print(f"Monte Carlo results saved to:")
for f in mc_files:
    print(f"  {f}")


In [ ]:
import re
def parse_action(raw_text):
    try:
        # Extract the first {...} block
        m = re.search(r"\{.*?\}", raw_text, flags=re.S)
        if not m:
            raise ValueError("No JSON object found")
        js_text = m.group()

        # Remove // comments (even inline)
        js_text = re.sub(r'//.*?(?=\n|$)', '', js_text)

        # Remove /* */ comments
        js_text = re.sub(r'/\*.*?\*/', '', js_text, flags=re.S)

        # Remove trailing commas before } or ]
        js_text = re.sub(r',\s*}', '}', js_text)
        js_text = re.sub(r',\s*\]', ']', js_text)

        # Replace smart quotes with regular quotes
        js_text = js_text.replace('\u201c', '"').replace('\u201d', '"')
        js_text = js_text.replace('\u2018', "'").replace('\u2019', "'")

        # Collapse multi-line strings (optional)
        js_text = re.sub(r'\n+', ' ', js_text)

        # Ensure keys are double-quoted (quick hack)
        js_text = re.sub(r'(\w+)\s*:', r'"\1":', js_text)

        # Strip whitespace
        js_text = js_text.strip()

        return json.loads(js_text)

    except Exception as e:
        print("COULD NOT PARSE JSON:", e)
        print(raw_text)
        return {"Reasoning": raw_text, "Action": "PLAY", "Declared_Rank": None, "Card_idx": []}


In [ ]:
import json
raw_text = """{
"Reasoning": "B played 0 cards claiming rank None. Since B's hand is 5 cards and they played 0, it is likely they are lying. I'm challenging to test the truth.",
"Action": "Challenge",
"Card_idx": []
}"""

parse_action(raw_text)

{'Reasoning': "B played 0 cards claiming rank None. Since B's hand is 5 cards and they played 0, it is likely they are lying. I'm challenging to test the truth.",
 'Action': 'Challenge',
 'Card_idx': []}